In [1]:
# WB JG 2/1/16
# batch downloads genbank protein descriptions + organism for antibiotic resistance genes

# imports
import pickle
import time
import sys
from Bio import SeqIO
from Bio import Entrez

# 
Entrez.email = 'morganleepetrovich@gmail.com'
 
# paths:
accessions_textfile = ''


In [4]:
# function definitions
def get_accessions(filepath):
    """ Takes a textfile with gi numbers in the second column and returns a list of GI numbers."""
    accession_numbers = []
    
    with open(filepath, 'r') as f:
        for line in f:
            pieces = line.split("\t")
            if pieces[1].startswith("GI"):
                gi = pieces[1][3:-1]
                accession_numbers.append(gi)
    
    return accession_numbers


def batch_prot_download(accession_list, start, stop):
    """Inputs: Accession list, range of values
    1. Retrieve name and organism from Genbank
    2. Add each (gi, name, organism) to a temporary dataframe 
    3. Append new dataframe to existing pickled dataframe, save and close it again
    """
    
    
    print "Downloading record %i to %i using efetch" %(start+1, stop)
    prot_data = {}
    stop = min(len(accession_list), stop) #stop early if end of accession_list is reached.
    
    for gi in accession_list[start:stop]:
        # Save each genbank entry as a dictionary so that it is expandable to multiple regions/CDS
        values = {}
        cds_count=0 
        region_count=0 
        
        # read from genbank
        handle = Entrez.efetch(db='protein', id = gi, rettype='gb', retmode='text')
        record = SeqIO.read(handle, 'gb')
        
        # Get description + organism.
        try:
            split =  record.description.split(" [")
            name = split[0]
            org = split[1][:-2]
            values['gi'] = gi
            values['name'] = name
            values['org'] = org
        except:
            pass
        
        
        # get any potentially relevant features
        for i in record.features:
            if i.type == "Region":
                try:
                    values['region_{0}'.format(region_count)] = {
                                                    'db_xref' : i.qualifiers['db_xref'], 
                                                     'region' : i.qualifiers['region_name'],
                                                     'note' : i.qualifiers['note']} 
                    region_count +=1
                except:
                    pass
    
            if i.type == "CDS":
                try:
                    keys = i.qualifiers.keys()
                    values["CD_{0}".format(cds_count)] = {}
                    for j in keys:
                        values["CD_{0}".format(cds_count)][j] = i.qualifiers[j]
                    cds_count +=1
                except:
                    pass
                
        prot_data[gi] = values
            
    return prot_data
        

def batch_processor(accession_list):
    """ Batch processes genbank database requests so they don't ban us
    Creates a pickled object after every 1000 requests in case it breaks before reaching 300,000 requests.
    Data is stored in a dictionary with keys = gi numbers and name, regions, CDs, and organism as values.
    I will probably need to convert to a dataframe later.
    """
    
    # Get size of accessions file
    total_size = len(accession_list)
    batch_size = 100
    
    # Create pickle
    pickle.dump({}, open("../Output/pickled.p", "wb"),2)
    
    # Batch process the genbank data
    last_known_good = {}
    for i in range(0, total_size, batch_size):
        last_known_good = pickle.load( open("../Output/pickled.p", "rb"))
        next_batch = batch_prot_download(accession_list, i, i+batch_size)
        last_known_good.update(next_batch)
        
        # Save updated version and sleep for 60 sec.
        pickle.dump(last_known_good, open("../Output/pickled.p", "wb"))
        #time.sleep(60) # 60?

    return

In [5]:
# Call functions from above
accession_list = get_accessions('/Users/jimbo/Downloads/accessions.txt')

# This needs to return something or save as  pickle
batch_processor(accession_list[:1000])


In [8]:
last_known_good = pickle.load( open("../Output/pickled.p", "rb"))
last_known_good['197213274']

# Get full structure as df?
# Or skip the regions from the CDD?

# How can I construct pandas dataframe row by row and add columns as necessary?
# Maybe SQL type database is better?


{'CD_0': {'coded_by': ['CP001138.1:830603..832900'],
  'locus_tag': ['SeAg_B0835'],
  'transl_table': ['11']},
 'gi': '197213274',
 'name': 'leucine-rich repeat protein',
 'org': 'Salmonella enterica subsp. enterica serovar Agona str. SL483',
 'region_0': {'db_xref': ['CDD:185268'],
  'note': ['E3 ubiquitin-protein ligase SlrP; Provisional'],
  'region': ['PRK15370']},
 'region_1': {'db_xref': ['CDD:260768'],
  'note': ['SGNH_hydrolase, or GDSL_hydrolase, is a diverse family of lipases and esterases. The tertiary fold of the enzyme is substantially different from that of the alpha/beta hydrolase family and unique among all known hydrolases; its active site closely...; cl01053'],
  'region': ['SGNH_hydrolase']},
 'region_10': {'db_xref': ['CDD:275380'],
  'note': ['leucine-rich repeat [structural motif]'],
  'region': ['leucine-rich repeat']},
 'region_11': {'db_xref': ['CDD:275380'],
  'note': ['leucine-rich repeat [structural motif]'],
  'region': ['leucine-rich repeat']},
 'region_12

## Description of genbank features

#Feature types and qualifiers dictionary values.
source: database id, organism, strain. 
protein: name - same as the decription 

Region will have a location and a cross-ref db, a note that could be useful,
Want to get db_xref, note, and region_name for every region. 
Save as values = {region_1:{db_xref: stuff, note: stuff, region_name: stuff}, region_2:{}

CDs will refer to coded_by == nucleotide accession, 
values = {CDS_1:{coded_by, note) db_xref: stuff, note: stuff, region_name: stuff}, region_2:{}

Possibly some sections are missing.
 